<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# LightGBM: A Highly Efficient Gradient Boosting Decision Tree
This notebook will give you an example of how to train a LightGBM model to estimate click-through rates on an e-commerce advertisement. We will train a LightGBM based model on the Criteo dataset.

[LightGBM](https://github.com/Microsoft/LightGBM) is a gradient boosting framework that uses tree-based learning algorithms. It is designed to be distributed and efficient with the following advantages:
* Fast training speed and high efficiency.
* Low memory usage.
* Great accuracy.
* Support of parallel and GPU learning.
* Capable of handling large-scale data.

## Global Settings and Imports

In [24]:
import sys
import os
import numpy as np
import lightgbm as lgb
import papermill as pm
import scrapbook as sb
import pandas as pd
import category_encoders as ce
from tempfile import TemporaryDirectory
from sklearn.metrics import roc_auc_score, log_loss

import recommenders.models.lightgbm.lightgbm_utils as lgb_utils
import recommenders.datasets.criteo as criteo

print("System version: {}".format(sys.version))
print("LightGBM version: {}".format(lgb.__version__))

System version: 3.7.13 (default, Mar 29 2022, 02:18:16) 
[GCC 7.5.0]
LightGBM version: 3.3.2


### Parameter Setting
Let's set the main related parameters for LightGBM now. Basically, the task is a binary classification (predicting click or no click), so the objective function is set to binary logloss, and 'AUC' metric, is used as a metric which is less effected by imbalance in the classes of the dataset.

Generally, we can adjust the number of leaves (MAX_LEAF), the minimum number of data in each leaf (MIN_DATA), maximum number of trees (NUM_OF_TREES), the learning rate of trees (TREE_LEARNING_RATE) and EARLY_STOPPING_ROUNDS (to avoid overfitting) in the model to get better performance.

Besides, we can also adjust some other listed parameters to optimize the results. [In this link](https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters.rst), a list of all the parameters is shown. Also, some advice on how to tune these parameters can be found [in this url](https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters-Tuning.rst). 

In [25]:
MAX_LEAF = 64
MIN_DATA = 20
NUM_OF_TREES = 100
TREE_LEARNING_RATE = 0.15
EARLY_STOPPING_ROUNDS = 20
METRIC = "auc"
SIZE = "sample"

In [26]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'num_class': 1,
    'objective': "binary",
    'metric': METRIC,
    'num_leaves': MAX_LEAF,
    'min_data': MIN_DATA,
    'boost_from_average': True,
    #set it according to your cpu cores.
    'num_threads': 24,
    'feature_fraction': 0.8,
    'learning_rate': TREE_LEARNING_RATE,
}

## Data Preparation
Here we use CSV format as the example data input. Our example data is a sample (about 100 thousand samples) from [Criteo dataset](https://www.kaggle.com/c/criteo-display-ad-challenge). The Criteo dataset is a well-known industry benchmarking dataset for developing CTR prediction models, and it's frequently adopted as evaluation dataset by research papers. The original dataset is too large for a lightweight demo, so we sample a small portion from it as a demo dataset.

Specifically, there are 39 columns of features in Criteo, where 13 columns are numerical features (I1-I13) and the other 26 columns are categorical features (C1-C26).

In [27]:
#posters = pd.read_csv("../01_prepare_data/ml-25m/poster-urls.csv")
#ratings = pd.read_csv("../01_prepare_data/ml-25m/ratings.csv")
#all_data = ratings.join(posters, on="movieId", lsuffix="_ratings", rsuffix="_posters")
all_data = pd.read_csv("../../carve/datasets/carve-movielens-prepared.csv")
all_data = all_data.drop("Unnamed: 0", axis=1)
all_data = all_data.sample(n=100000, random_state=0)
all_data.reset_index(inplace=True)
all_data = all_data.drop("index", axis=1)

display(all_data.head())

/home/baki/anaconda/envs/carve37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (6,7,8,9,10,11,12,14,15,17,18,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,movieId,userId,rating,timestamp,genres_0,genres_1,genres_2,genres_3,genres_4,categories_0,...,categories_2,color_0,color_1,color_2,tags_0,tags_1,tags_2,description_0,description_1,description_2
0,5418,50997.0,5.0,1.198447e+09,Action,Mystery,Thriller,NaN,NaN,others_,...,NaN,Orange,Black,NaN,human face,text,poster,text,NaN,NaN
1,44191,60716.0,4.0,1.437543e+09,Action,Sci-Fi,Thriller,IMAX,NaN,people_show,...,NaN,Brown,Red,Black,book,human face,album cover,candle,text,dark
2,60069,1691.0,5.0,1.534741e+09,Adventure,Animation,Children,Romance,Sci-Fi,text_mag,...,NaN,Black,Blue,NaN,text,screenshot,cartoon,calendar,NaN,NaN
3,1376,152325.0,4.0,9.755375e+08,Adventure,Comedy,Sci-Fi,NaN,NaN,people_,...,NaN,Black,Pink,Yellow,book,painting,human face,text,book,NaN
4,608,158007.0,4.0,8.320073e+08,Comedy,Crime,Drama,Thriller,NaN,text_sign,...,NaN,White,NaN,NaN,stitch,text,embroidery,text,NaN,NaN


In [28]:
all_data.shape

(100000, 21)

In [29]:
print(all_data.isnull().sum())
#all_data = all_data.dropna()

movieId              0
userId               0
rating               0
timestamp            0
genres_0             0
genres_1         16163
genres_2         45161
genres_3         77107
genres_4         93075
categories_0      4942
categories_1     50793
categories_2     83622
color_0              0
color_1          34867
color_2          83101
tags_0              12
tags_1             438
tags_2            1303
description_0      730
description_1    30643
description_2    58035
dtype: int64


First, we cut three sets (train_data (first 80%), valid_data (middle 10%) and test_data (last 10%)), cut from the original all data. <br>
Notably, considering the Criteo is a kind of time-series streaming data, which is also very common in recommendation scenario, we split the data by its order.

In [30]:
all_data.columns

Index(['movieId', 'userId', 'rating', 'timestamp', 'genres_0', 'genres_1',
       'genres_2', 'genres_3', 'genres_4', 'categories_0', 'categories_1',
       'categories_2', 'color_0', 'color_1', 'color_2', 'tags_0', 'tags_1',
       'tags_2', 'description_0', 'description_1', 'description_2'],
      dtype='object')

In [31]:
# split data to 3 sets    
length = len(all_data)
train_data = all_data.loc[:0.8*length-1]
valid_data = all_data.loc[0.8*length:0.9*length-1]
test_data = all_data.loc[0.9*length:]

movieId                  6104
userId                26203.0
rating                    5.0
timestamp        1112973160.0
genres_0               Comedy
genres_1                  NaN
genres_2                  NaN
genres_3                  NaN
genres_4                  NaN
categories_0         text_mag
categories_1              NaN
categories_2              NaN
color_0                 Green
color_1                 Black
color_2                   NaN
tags_0                   book
tags_1                   text
tags_2                 poster
description_0            text
description_1             NaN
description_2             NaN
Name: 70000, dtype: object
(100000, 21)


## Basic Usage
### Ordinal Encoding
Considering LightGBM could handle the low-frequency features and missing value by itself, for basic usage, we only encode the string-like categorical features by an ordinal encoder.

In [32]:
cate_cols = ["userId", "movieId", 
    "genres_0", "genres_1", "genres_2", "genres_3", "genres_4", "categories_0", "categories_1", "categories_2", "color_0", "color_1", "color_2", "tags_0", "tags_1", "tags_2", "description_0", "description_1", "description_2"]
label_col = "rating"

ord_encoder = ce.ordinal.OrdinalEncoder(cols=cate_cols)

def encode_csv(df, encoder, label_col, typ='fit'):
    if typ == 'fit':
        df = encoder.fit_transform(df)
    else:
        df = encoder.transform(df)
    y = df[label_col].values
    del df[label_col]
    return df, y

train_x, train_y = encode_csv(train_data, ord_encoder, label_col)
valid_x, valid_y = encode_csv(valid_data, ord_encoder, label_col, 'transform')
test_x, test_y = encode_csv(test_data, ord_encoder, label_col, 'transform')

print('Train Data Shape: X: {trn_x_shape}; Y: {trn_y_shape}.\nValid Data Shape: X: {vld_x_shape}; Y: {vld_y_shape}.\nTest Data Shape: X: {tst_x_shape}; Y: {tst_y_shape}.\n'
      .format(trn_x_shape=train_x.shape,
              trn_y_shape=train_y.shape,
              vld_x_shape=valid_x.shape,
              vld_y_shape=valid_y.shape,
              tst_x_shape=test_x.shape,
              tst_y_shape=test_y.shape,))
train_x.head()

Train Data Shape: X: (80000, 20); Y: (80000,).
Valid Data Shape: X: (10000, 20); Y: (10000,).
Test Data Shape: X: (10000, 20); Y: (10000,).



,movieId,userId,timestamp,genres_0,genres_1,genres_2,genres_3,genres_4,categories_0,categories_1,categories_2,color_0,color_1,color_2,tags_0,tags_1,tags_2,description_0,description_1,description_2
0,1,1,1.198447e+09,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,2,2,1.437543e+09,1,2,1,2,1,2,2,1,2,2,2,2,2,2,2,2,2
2,3,3,1.534741e+09,2,3,2,3,2,3,2,1,3,3,1,3,3,3,3,1,1
3,4,4,9.755375e+08,2,4,3,1,0,4,2,0,3,4,3,2,4,4,1,3,0
4,5,5,8.320073e+08,3,5,4,4,1,5,3,1,4,5,0,4,1,5,1,1,1


### Create model
When both hyper-parameters and data are ready, we can create a model:

In [33]:
lgb_train = lgb.Dataset(train_x, train_y.reshape(-1), params=params, categorical_feature=cate_cols)
lgb_valid = lgb.Dataset(valid_x, valid_y.reshape(-1), reference=lgb_train, categorical_feature=cate_cols)
lgb_test = lgb.Dataset(test_x, test_y.reshape(-1), reference=lgb_train, categorical_feature=cate_cols)
lgb_model = lgb.train(params,
                      lgb_train,
                      num_boost_round=NUM_OF_TREES,
                      early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                      valid_sets=lgb_valid,
                      categorical_feature=cate_cols)

[LightGBM] [Warning] Contains only one class
[LightGBM] [Info] Number of positive: 80000, number of negative: 0
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002479 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12837
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=1.000000 -> initscore=34.539576
[LightGBM] [Info] Start training from score 34.539576
[LightGBM] [Info] [binary:BoostFromScore]: pavg=1.000000 -> initscore=34.539576
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[1]	valid_0's auc: 1
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[2]	valid_0's auc: 1
[LightGBM] [Warning] Stopped training because there are no more leaves

/home/baki/anaconda/envs/carve37/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/baki/anaconda/envs/carve37/lib/python3.7/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/baki/anaconda/envs/carve37/lib/python3.7/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Now let's see what is the model's performance:

In [35]:
"""
test_preds = lgb_model.predict(test_x)
auc = roc_auc_score(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))
logloss = log_loss(np.asarray(test_y.reshape(-1)), np.asarray(test_preds), eps=1e-12)
res_basic = {"auc": auc, "logloss": logloss}
print(res_basic)
sb.glue("res_basic", res_basic)
"""

'\ntest_preds = lgb_model.predict(test_x)\nauc = roc_auc_score(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))\nlogloss = log_loss(np.asarray(test_y.reshape(-1)), np.asarray(test_preds), eps=1e-12)\nres_basic = {"auc": auc, "logloss": logloss}\nprint(res_basic)\nsb.glue("res_basic", res_basic)\n'

<script type="text/javascript" src="http://cdn.mathjax.org/mathjax/latest/MathJax.js?config=default"></script>
## Optimized Usage
### Label-encoding and Binary-encoding
Next, since LightGBM has a better capability in handling dense numerical features effectively, we try to convert all the categorical features in original data into numerical ones, by label-encoding [3] and binary-encoding [4]. Also due to the sequence property of Criteo, the label-encoding we adopted is executed one-by-one, which means we encode the samples in order, by the information of the previous samples before each sample (sequential label-encoding and sequential count-encoding). Besides, we also filter the low-frequency categorical features and fill the missing values by the mean of corresponding columns for the numerical features. (consulting `lgb_utils.NumEncoder`)

Specifically, in `lgb_utils.NumEncoder`, the main steps are as follows.
* Firstly, we convert the low-frequency categorical features to `"LESS"` and the missing categorical features to `"UNK"`. 
* Secondly, we convert the missing numerical features into the mean of corresponding columns. 
* Thirdly, the string-like categorical features are ordinal encoded like the example shown in basic usage. 
* And then, we target encode the categorical features in the samples order one-by-one. For each sample, we add the label and count information of its former samples into the data and produce new features. Formally, for $i=1,2,...,n$, we add $\frac{\sum\nolimits_{j=1}^{i-1} I(x_j=c) \cdot y}{\sum\nolimits_{j=1}^{i-1} I(x_j=c)}$ as a new label feature for current sample $x_i$, where $c$ is a category to encode in current sample, so $(i-1)$ is the number of former samples, and $I(\cdot)$ is the indicator function that check the former samples contain $c$ (whether $x_j=c$) or not. At the meantime, we also add the count frequency of $c$, which is $\frac{\sum\nolimits_{j=1}^{i-1} I(x_j=c)}{i-1}$, as a new count feature. 
* Finally, based on the results of ordinal encoding, we add the binary encoding results as new columns into the data.

Note that the statistics used in the above process only updates when fitting the training set, while maintaining static when transforming the testing set because the label of test data should be considered as unknown.

In [36]:
label_col = 'rating'
nume_cols = []
num_encoder = lgb_utils.NumEncoder(cate_cols, nume_cols, label_col)
train_x, train_y = num_encoder.fit_transform(train_data)
valid_x, valid_y = num_encoder.transform(valid_data)
test_x, test_y = num_encoder.transform(test_data)
del num_encoder
print('Train Data Shape: X: {trn_x_shape}; Y: {trn_y_shape}.\nValid Data Shape: X: {vld_x_shape}; Y: {vld_y_shape}.\nTest Data Shape: X: {tst_x_shape}; Y: {tst_y_shape}.\n'
      .format(trn_x_shape=train_x.shape,
              trn_y_shape=train_y.shape,
              vld_x_shape=valid_x.shape,
              vld_y_shape=valid_y.shape,
              tst_x_shape=test_x.shape,
              tst_y_shape=test_y.shape,))


2022-05-08 20:43:35,093 [INFO] Filtering and fillna features
100%|██████████| 19/19 [00:01<00:00, 10.95it/s]
0it [00:00, ?it/s]
2022-05-08 20:43:36,833 [INFO] Ordinal encoding cate features
2022-05-08 20:43:37,420 [INFO] Target encoding cate features
100%|██████████| 19/19 [00:01<00:00, 11.55it/s]
2022-05-08 20:43:39,068 [INFO] Start manual binary encoding
100%|██████████| 19/19 [00:01<00:00, 12.10it/s]
2022-05-08 20:43:43,505 [INFO] Filtering and fillna features
100%|██████████| 19/19 [00:00<00:00, 416.94it/s]
0it [00:00, ?it/s]
2022-05-08 20:43:43,554 [INFO] Ordinal encoding cate features
2022-05-08 20:43:43,613 [INFO] Target encoding cate features
100%|██████████| 19/19 [00:00<00:00, 90.48it/s]
2022-05-08 20:43:43,825 [INFO] Start manual binary encoding
100%|██████████| 19/19 [00:01<00:00, 12.71it/s]
2022-05-08 20:43:48,098 [INFO] Filtering and fillna features
100%|██████████| 19/19 [00:00<00:00, 377.38it/s]
0it [00:00, ?it/s]
2022-05-08 20:43:48,152 [INFO] Ordinal encoding cate fea

Train Data Shape: X: (80000, 156); Y: (80000, 1).
Valid Data Shape: X: (10000, 156); Y: (10000, 1).
Test Data Shape: X: (10000, 156); Y: (10000, 1).



### Training and Evaluation

In [37]:
lgb_train = lgb.Dataset(train_x, train_y.reshape(-1), params=params)
lgb_valid = lgb.Dataset(valid_x, valid_y.reshape(-1), reference=lgb_train)
lgb_model = lgb.train(params,
                      lgb_train,
                      num_boost_round=NUM_OF_TREES,
                      early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                      valid_sets=lgb_valid)

/home/baki/anaconda/envs/carve37/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Contains only one class
[LightGBM] [Info] Number of positive: 80000, number of negative: 0
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010393 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9921
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 154
[LightGBM] [Info] [binary:BoostFromScore]: pavg=1.000000 -> initscore=34.539576
[LightGBM] [Info] Start training from score 34.539576
[LightGBM] [Info] [binary:BoostFromScore]: pavg=1.000000 -> initscore=34.539576
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[1]	valid_0's auc: 1
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[2]	valid_0's auc: 1
[Light

In [39]:
"""
test_preds = lgb_model.predict(test_x)
auc = roc_auc_score(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))
logloss = log_loss(np.asarray(test_y.reshape(-1)), np.asarray(test_preds), eps=1e-12)
res_optim = {"auc": auc, "logloss": logloss}
print(res_optim)
sb.glue("res_optim", res_optim)
"""

'\ntest_preds = lgb_model.predict(test_x)\nauc = roc_auc_score(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))\nlogloss = log_loss(np.asarray(test_y.reshape(-1)), np.asarray(test_preds), eps=1e-12)\nres_optim = {"auc": auc, "logloss": logloss}\nprint(res_optim)\nsb.glue("res_optim", res_optim)\n'

## Model saving and loading
Now we finish the basic training and testing for LightGBM, next let's try to save and reload the model, and then evaluate it again.

In [41]:
with TemporaryDirectory() as tmp:
    save_file = os.path.join(tmp, r'finished.model')
    lgb_model.save_model(save_file)
    loaded_model = lgb.Booster(model_file=save_file)

# eval the performance again
test_preds = loaded_model.predict(test_x)

"""
auc = roc_auc_score(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))
logloss = log_loss(np.asarray(test_y.reshape(-1)), np.asarray(test_preds), eps=1e-12)
print({"auc": auc, "logloss": logloss})
"""

'\nauc = roc_auc_score(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))\nlogloss = log_loss(np.asarray(test_y.reshape(-1)), np.asarray(test_preds), eps=1e-12)\nprint({"auc": auc, "logloss": logloss})\n'

## Additional Reading

\[1\] Guolin Ke, Qi Meng, Thomas Finley, Taifeng Wang, Wei Chen, Weidong Ma, Qiwei Ye, and Tie-Yan Liu. 2017. LightGBM: A highly efficient gradient boosting decision tree. In Advances in Neural Information Processing Systems. 3146–3154.<br>
\[2\] The parameters of LightGBM: https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters.rst <br>
\[3\] Anna Veronika Dorogush, Vasily Ershov, and Andrey Gulin. 2018. CatBoost: gradient boosting with categorical features support. arXiv preprint arXiv:1810.11363 (2018).<br>
\[4\] Scikit-learn. 2018. categorical_encoding. https://github.com/scikit-learn-contrib/categorical-encoding<br>
